In [3]:
# !pip install pdfplumber
# !pip install docx
# !pip install docx2txt
# !pip install gradio
# !pip install transformers
# !pip install torch
# !pip install accelerate
# !pip install sentencepiece
# !pip install torch
# !pip install pypdf
# !pip install speechrecognition
# !pip install scikit-learn
#!pip install gtts

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# -*- coding: utf-8 -*-
"""app.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1BdLeypcUPTiU4DdVy8PrlXs--8wXRHog
"""

# !pip install pdfplumber
# !pip install docx
# !pip install docx2txt
# !pip install gradio
# !pip install transformers
# !pip install torch
# !pip install accelerate
# !pip install sentencepiece
# !pip install torch
# !pip install pypdf

import gradio as gr
import pdfplumber
import docx2txt
import os
import re
import torch
import csv
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import speech_recognition as sr
import pandas as pd
from gtts import gTTS


# Set the GRADIO_TEMP_DIR to a location with write permissions (e.g., current directory)
os.environ['GRADIO_TEMP_DIR'] = './gradio_temp'

# Ensure the directory exists
if not os.path.exists('./gradio_temp'):
    os.makedirs('./gradio_temp')


# Define the model and tokenizer
model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    load_in_4bit=True,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)



def s2t(filepath):
  recognizer = sr.Recognizer()

  with sr.AudioFile(filepath) as source:
      audio = recognizer.record(source)

  text = recognizer.recognize_google(audio)
  print(text)
  return text

def tts_func(recent_question):
    language = 'en'
    tts = gTTS(text=recent_question, lang=language, slow=False)
    output_file = "./gradio_temp/output.wav"
    tts.save(output_file)
    print(f"Audio file saved as {output_file}")
    return output_file
    

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using pdfplumber."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def extract_text_from_docx(docx_path):
    """Extract text from a DOCX file using docx2txt."""
    text = docx2txt.process(docx_path)
    return text

def extract_resume_info(text):
    """Extract key information from the resume text."""
    print("Extracting resume information...")
    text = text.lower()
    headers = {
        'name': 'NAME',
        'skills': 'SKILLS',
        'experience': 'EXPERIENCE',
        'education': 'EDUCATION'
    }
    first_line = text.split('\n', 1)[0].strip()
    name_match = re.match(r'^.*\s*name\s*[:\-]?\s*(.*)', first_line, re.IGNORECASE)
    name = name_match.group(1).strip() if name_match else first_line
    print(f"Extracted Name: {name}")

    for key, value in headers.items():
        text = re.sub(rf'\b{key}\b', value, text, flags=re.IGNORECASE)

    def extract_section(header):
        pattern = rf'\b{header}\b\s*[:\-]?\s*(.*?)(?=\n[A-Z]|$)'
        match = re.search(pattern, text, re.DOTALL)
        result = match.group(1).strip() if match else 'Not Found'
        print(f"Extracted {header}: {result}")
        return result

    skills = extract_section('SKILLS')
    experience = extract_section('EXPERIENCE')
    education = extract_section('EDUCATION')

    # Ensure experience is treated as an integer for comparison
    experience_present = 1 if experience != 'Not Found' else 0
    return name, skills, experience_present, education

def generate_response(prompt):
    """Generate a response using the language model."""
    print(f"Generating response for prompt: {prompt}")
    generation_args = {
        "max_new_tokens": 90,
        "return_full_text": True,
        "temperature": 0.7,
        "do_sample": True,
    }
    messages = [ 
        {"role": "system", "content": "You are a job interviewer assistant. You speak in an empathetic way with human like expressions such as: 'hi', 'hello', 'Good Morning', 'How are you', 'oh'. 'ok', 'fine', 'that's great', 'hmm' and so on"}, 
        {"role": "user", "content": prompt}, 
    ] 
    pipe = pipeline( 
        "text-generation", 
        model=model, 
        tokenizer=tokenizer, 
    ) 
    output = pipe(messages, **generation_args)
    a = output[0]['generated_text']
    generated_text = next(item['content'] for item in a if item['role'] == 'assistant')

    # Find all questions that start with a number or with "Question:"
    # questions = re.findall(r'(?:\d+\.|\b[qQ]uestion:).*?\?', generated_text)

    print(f"Generated Text: {generated_text}")

    questions = re.findall(r'(?:\d+\.|\b[qQ]uestion:).*?\?', generated_text)
    # Return the last question if any, otherwise return a default message
    return questions[-1] if questions else "No relevant question generated."
    # return generated_text

def generate_response_for_df(df):
    """Generate model responses for each question in the DataFrame."""
    # List to store model responses
    model_responses = []

    for i, row in df.iterrows():
        question = row['question']
        print(f"Generating response for question: {question}")

        # Generate a response using the existing generate_response logic
        generation_args = {
            "max_new_tokens": 90,
            "return_full_text": True,
            "temperature": 0.7,
            "do_sample": True,
        }
        output = pipe(question, **generation_args)
        generated_text = output[0]['generated_text']

        # Directly use the generated response
        model_response = generated_text

        print(f"Model Response: {model_response}")
        model_responses.append(model_response)

    # Add the generated model responses to the DataFrame
    df['model_response'] = model_responses
    return df
def calculate_similarity(df):
    """Calculate similarity between user and model responses and return updated DataFrame."""
    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    similarity_scores = []

    for i, row in df.iterrows():
        user_response = row['user_response']
        model_response = row['model_response']

        # Compute TF-IDF for the user response and model response
        tfidf_matrix = vectorizer.fit_transform([user_response, model_response])

        # Calculate cosine similarity between user and model response
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        # Convert similarity to percentage
        similarity_percentage = round(similarity * 100, 2)

        print(f"Similarity score for row {i}: {similarity_percentage}%")
        similarity_scores.append(similarity_percentage)

    # Add the similarity scores to the DataFrame
    df['similarity'] = similarity_scores
    return df

def process_resume(file_path):
    """Process the resume to extract text and key information."""
    print(f"Processing resume file: {file_path}")
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.pdf':
        extracted_text = extract_text_from_pdf(file_path)
    elif file_extension == '.docx':
        extracted_text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a PDF or DOCX file.")
    print(f"Extracted Text: {extracted_text[:200]}...")  # Show a snippet of the extracted text
    return extracted_text



def run_interview(name, skills, experience, user_response=None, questions_and_answers=None, user=None, stage=0):
    """Conduct an interview based on the extracted resume data."""
    if questions_and_answers is None:
        questions_and_answers = []
    if user is None:
        user = []


    print(f"Stage: {stage}")
    print(f"User Response: {user_response}")
    print(questions_and_answers)
    print("User", user)

    if experience and skills:
        if stage == 0:
            print("step: 0")
            experience_question = f"Hi there! Good Morning. Let's Start the interview. Can you describe a challenging project where you applied your experience?"
            questions_and_answers.append([experience_question, ""])
            print(f"Stage 0: Added experience question: {experience_question}")
            stage += 1

        elif stage == 1 and user_response:
            print("step: 1")
            response= s2t(user_response)
            user.append(response)
            print(user)
            prompt = f"Please generate only 1 follow-up question starting with 'Question:', WITHOUT any additional text or answer...just the question.. based on this: {response}"
            follow_up_question = generate_response(prompt)
            questions_and_answers.append([follow_up_question, ""])
            print(f"Stage 1: Added follow-up question: {follow_up_question}")
            stage += 1

        elif stage == 2 and user_response:
            # Ensure questions_and_answers is empty at the beginning of stage 2
                print ("step2_1")
                response= s2t(user_response)
                user.append(response)
                prompt_for_questions = f"Generate 1 interview questions based on the following skills: {skills}. Please provide only the questions focused on the subject's bookish knowledge, with no headings or additional text."
                generated_question = generate_response(prompt_for_questions)
                questions_and_answers.append([generated_question,""])
                stage += 1
        elif stage == 3 and user_response:
                print ("step3")
                response= s2t(user_response)
                user.append(response)
                prompt_for_questions = f"Generate 1 interview questions based on the following skills: {skills} not this question {questions_and_answers[-1][0]}. Please provide only the questions focused on the subject's bookish knowledge, with no headings or additional text."
                generated_question = generate_response(prompt_for_questions)
                questions_and_answers.append([generated_question,""])
                stage += 1
        elif stage == 4 and user_response:
                print ("step4")
                response= s2t(user_response)
                user.append(response)
                prompt_for_questions = f"Generate 1 interview questions based on the following skills: {skills} not this question {questions_and_answers[-1][0]}. Please provide only the questions focused on the subject's bookish knowledge, with no headings or additional text."
                generated_question = generate_response(prompt_for_questions)
                questions_and_answers.append([generated_question,""])
                stage += 1
        elif stage == 5 and user_response:
                print ("step5")
                response= s2t(user_response)
                user.append(response)
                prompt_for_questions = f"Generate 1 interview questions based on the following skills: {skills} not this question {questions_and_answers[-1][0]}. Please provide only the questions focused on the subject's bookish knowledge, with no headings or additional text."
                generated_question = generate_response(prompt_for_questions)
                questions_and_answers.append([generated_question,""])
                stage += 1

        elif stage == 6 and user_response:
            print("Stage 6: Adding user responses and updating stage...")
            response= s2t(user_response)
            user.append(response)
            print("user response1", user)
            print("q nd a", questions_and_answers)
            print("user response2", user)
            questions_and_answers = [q[0] for q in questions_and_answers]

            # Create DataFrame
            df = pd.DataFrame({
                'question': questions_and_answers,
                'user_response': user
            })
            # df = pd.DataFrame(questions_and_answers, columns=["Question", "User_Response"])
            df = df.drop(index=[0, 1])
            df= generate_response_for_df(df)
            df=calculate_similarity(df)
            df.to_csv("skills_interview_results.csv", index=False)
            print(f"DataFrame saved to CSV.")
            questions_and_answers.append(["Ok, got it... Thanks for taking the interview. Have a great day", ""])
            # print("Exitting")
            # sys.exit("Stopping the script")
            stage += 1

        # elif stage == 7:
        #     print("Stage 7: Saving skills-related Q&A to CSV...")
        #     df = pd.DataFrame(questions_and_answers, columns=["Question", "User Response"])
        #     df.to_csv("skills_interview_results.csv", index=False)
        #     print(f"DataFrame saved to CSV.")

        #     # Resetting the state
        #     questions_and_answers = []
        #     print("Exitting")
        #     sys.exit("Stopping the script")
        #     stage += 1

    # Return only the most recent question for display and the updated stage
    recent_question = questions_and_answers[-1][0] if questions_and_answers else ""
    recq= tts_func(recent_question)
    print(f"Returning recent question: {recent_question}")
    return recq, stage, questions_and_answers,user


def upload_resume(file_path):
    """Upload and process the resume to extract key information."""
    print(f"Uploading resume file: {file_path}")
    resume_text = process_resume(file_path)
    name, skills, experience, education = extract_resume_info(resume_text)
    return name, skills, experience, education, resume_text

def interview_interface(name, skills, experience, user_response=None,stage=0, questions_and_answers=None, user= None):
    print(user_response)
    recent_question, new_stage, updated_questions_and_answers, updated_user = run_interview(name, skills, experience, user_response, questions_and_answers, user, stage)
    return recent_question, new_stage, updated_questions_and_answers, updated_user

def start_interview_from_file(file):
    """Extract resume info and automatically start the interview."""
    name, skills, experience, education, _ = upload_resume(file.name)
    return interview_interface(name, skills, experience, stage=0, questions_and_answers=[], user=[])

with gr.Blocks() as demo:
    with gr.Tabs():
        with gr.TabItem("Upload Resume"):
            resume_file = gr.File(label="Upload your resume (PDF or DOCX):")
            name_output = gr.Textbox(label="Name", interactive=False)
            skills_output = gr.Textbox(label="Skills", interactive=False)
            experience_output = gr.Textbox(label="Experience", interactive=False)
            education_output = gr.Textbox(label="Education", interactive=False)
            process_button = gr.Button("Process Resume")
            process_button.click(upload_resume, inputs=resume_file, outputs=[name_output, skills_output, experience_output, education_output])

        with gr.TabItem("Interview"):
            recent_question_output = gr.Audio(label="Colloquium says",type="filepath")
            user_response_input = gr.Audio(label="What do you think", type="filepath")
            submit_response_button = gr.Button("Answer now")

            # Initialize interview state and questions list
            stage = gr.State(value=0)
            questions_and_answers = gr.State(value=[])
            user = gr.State(value=[])
            # Automatically start interview when the resume is processed
            resume_file.change(start_interview_from_file, inputs=resume_file, outputs=[recent_question_output, stage, questions_and_answers, user])

            # Capture user response, generate model response, and update stage
            submit_response_button.click(
                interview_interface,
                inputs=[name_output, skills_output, experience_output, user_response_input, stage, questions_and_answers, user],
                outputs=[recent_question_output, stage, questions_and_answers, user]
            )

demo.launch(debug ='True', share='True')



`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0ba61fce90bd6a4352.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Uploading resume file: /home/rit/rit/research/gradio_temp/6d4a08df123d033759cf3cb5d0b8a99bbced0133e30d949ee520e6269f8d88bf/kumar-pallavs-one-page-resume.pdf
Processing resume file: /home/rit/rit/research/gradio_temp/6d4a08df123d033759cf3cb5d0b8a99bbced0133e30d949ee520e6269f8d88bf/kumar-pallavs-one-page-resume.pdf
Extracted Text: Kumar Pallav Email: me@kumarpallav.com
http://kumarpallav.com Mobile: +1-206-910-0006
EDUCATION
Indian Institute of Technology, Bombay Mumbai,India
BachelorofComputerScienceandEngineering(withHons.);C...
Extracting resume information...
Extracted Name: kumar pallav email: me@kumarpallav.com
Extracted SKILLS: languages: c++,csharp,javascript,java,c technologies: nodejs,uwp,win32
Extracted EXPERIENCE: microsoft redmond,wa
softwareengineer·onenote jun2016-present
◦ magic ink and ink lookup: workedonrecognizinginkstrokesintowords(usingonboardia)andrecognizing
keywords/entitiesgiventhecontext(usingcustomentityrecognitionservice). implementedaccumulationofimagesor
in

/opt/conda/envs/rit_fnc/lib/python3.10/site-packages/gradio/blocks.py:1776: UserWarning: A function (upload_resume) returned too many output values (needed: 4, returned: 5). Ignoring extra values.
    Output components:
        [textbox, textbox, textbox, textbox]
    Output values returned:
        ["kumar pallav email: me@kumarpallav.com", "languages: c++,csharp,javascript,java,c technologies: nodejs,uwp,win32", 1, "indian institute of technology, bombay mumbai,india
bachelorofcomputerscienceandengineering(withhons.);cgpa8.6/10 jun2010-may2014", "Kumar Pallav Email: me@kumarpallav.com
http://kumarpallav.com Mobile: +1-206-910-0006
EDUCATION
Indian Institute of Technology, Bombay Mumbai,India
BachelorofComputerScienceandEngineering(withHons.);CGPA8.6/10 Jun2010-May2014
EXPERIENCE
Microsoft Redmond,WA
SoftwareEngineer·OneNote Jun2016-Present
◦ Magic Ink and Ink Lookup: Workedonrecognizinginkstrokesintowords(usingonboardIA)andrecognizing
keywords/entitiesgiventhecontext(usingcustomentit